## Model Deployment

### 1. Import Azure Machine Learning Module

In [1]:
# pip install azureml-defaults
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
print(azureml.core.VERSION)

1.0.21


### 2. Create Azure ML Workspace

In [5]:
AZ_SUBSCRIPTION_ID="Your_Azure_Subscription_ID"

In [6]:
ws = Workspace.create(name='fraud_model',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='mi2',
                      create_resource_group=True,
                      location='southeastasia'
                     )

Deploying KeyVault with name fraudmodkeyvaultfimrckpx.
Deploying StorageAccount with name fraudmodstorageedfrvpxl.
Deploying AppInsights with name fraudmodinsightsveqpdqcq.
Deployed AppInsights with name fraudmodinsightsveqpdqcq.
Deploying ContainerRegistry with name fraudmodacrkqbbqvel.
Deployed ContainerRegistry with name fraudmodacrkqbbqvel.
Deployed KeyVault with name fraudmodkeyvaultfimrckpx.
Deployed StorageAccount with name fraudmodstorageedfrvpxl.
Deploying Workspace with name fraud_model.
Deployed Workspace with name fraud_model.


### 3. Save Configuration to Local

In [7]:
ws.write_config()

Wrote the config file config.json to: /Users/abhishekverma/Applications/fraud-detection/aml_config/config.json


### 4. Register ML Model to Azure

In [8]:
model = Model.register(model_path = "output/fraud_model.pkl",
                       model_name = "fraud_model",
                       tags = {"key": "0.1"},
                       description = "Fraud detection prediction",
                       workspace = ws)

Registering model fraud_model


### 5. Define Deploymemt Configuration for Azure

In [9]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data":"PS Data",
                                                     "method":"sklearn",
                                                     "type":"classification"}, 
                                               description='Fraud detection prediction')

### 6. Define Enviroment Configuration File

In [10]:
pythonConfig = CondaDependencies()
pythonConfig.add_conda_package("scikit-learn")

In [11]:
with open("pythonConfig.yml","w") as file:
    file.write(pythonConfig.serialize_to_string())

In [12]:
with open("pythonConfig.yml","r") as file:
    print(file.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- scikit-learn



### 7. Create Azure ML Scoring File (Execution Script)

In [13]:
%%writefile score.py
import numpy as np
import json, os, pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('fraud_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Writing score.py


### 8. Deploy Model to Azure

In [14]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="pythonConfig.yml")

CPU times: user 829 µs, sys: 1.06 ms, total: 1.89 ms
Wall time: 974 µs


### 9. Expose web service

In [15]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='fraud-detection-webservice',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image fraud-detection-webservice:1, operation "Succeeded"
Creating service
Running..................
SucceededACI service creation operation finished, operation "Succeeded"


In [16]:
print("Service is running on {}".format(service.scoring_uri))

Service is running on http://40.65.169.123:80/score


### 9. Use web service

In [18]:
!curl -X POST \
	-H 'Content-Type':'application/json' \
	-d '{"data":[[1,2,181.00,181.0,0.00,0.0,0.0]]}' \
	http://40.65.169.123:80/score

"[1]"

### 10. Delete the web service

In [19]:
ws.delete()